In [ ]:
#Importamos las librerias
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet
import snscrape.modules.twitter as sntwitter
import plotly.express as px
import neattext as nt
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from collections import Counter

In [ ]:
#Cargamos las stopwords de nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Importamos los datos de twitter
query = "#KingsLeagueJ5"
tweets = []
limit = 10000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.content, tweet.hashtags, tweet.username])

<ipython-input-10-190beaf53b2f>:13: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.content, tweet.hashtags, tweet.username])
<ipython-input-10-190beaf53b2f>:13: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.content, tweet.hashtags, tweet.username])


In [ ]:
#Colocamos los datos en un dataframe
dt = pd.DataFrame(tweets, columns=['Date', 'Tweet', 'Hashtag', 'Username'])
dt.head()

,Date,Tweet,Hashtag,Username
0,2023-02-09 19:00:39+00:00,🕵🏻 ¿QUIÉN ES? #KingsLeague 👑\n\n🔍 Pistas:\n- ...,"[KingsLeague, Fantasy, Juego, KingsLeagueJ5, K...",KingsLeagueFM
1,2023-02-09 15:22:57+00:00,"Por cierto, el partido entre Los Troncos y Ani...",[KingsLeagueJ5],Keliflogo
2,2023-02-09 15:06:55+00:00,HOY 🚨\nDirecto analizando la última jornada de...,"[KingsLeagueJ6, KingsLeaguej5]",CharlasDeVar
3,2023-02-09 14:51:58+00:00,Spursito y Rivers lo estaban pasando fatal.\n\...,[KingsLeagueJ5],Keliflogo
4,2023-02-09 14:27:46+00:00,La verdad es que la Haka de Gerard en el penal...,[KingsLeagueJ5],Keliflogo


In [ ]:
#Convertimos la columna Date a formato tiempo
dt['Date'] = pd.to_datetime(dt['Date'], format='%Y%m%d')

In [ ]:
#Filtamos los datos del periodo de tiempo en el cual se jugo la kings leage 
Df = dt[dt['Date'] > '2023-02-05']
DfToday = Df[Df['Date'] < '2023-02-06']

In [ ]:
df = DfToday

In [ ]:
df.head(2000)

,Date,Tweet,Hashtag,Username
85,2023-02-05 23:51:27+00:00,XBUYER TEAM de XBUYER VS 1K FC de IKER CASILLA...,[KingsLeagueJ5],jorgedelcuri
86,2023-02-05 23:50:07+00:00,"Que @IbaiLlanos , ni @auronplay , los directos...",[KingsLeagueJ5],dairony26
87,2023-02-05 23:32:18+00:00,Aniquiladores está para campeón?? #KingsLeagueJ5,[KingsLeagueJ5],Cristian_OuO
88,2023-02-05 23:25:37+00:00,"¡Fin del programa!\n\nUn placer, como siempre ...",[KingsLeagueJ5],KioscoKL
89,2023-02-05 23:24:48+00:00,💣 @Spursito hace SCOUTING de la Queens League ...,"[KingsLeagueJ5, QueensLeague]",KioscoKL
...,...,...,...,...
1497,2023-02-05 08:50:36+00:00,"@JijantesFC hoy es vuestro día, la primera vic...",[KingsLeaguej5],El_joaky_
1498,2023-02-05 05:00:00+00:00,¡MATCH DAY!🦁🖤\n\nNos vemos hoy ⏰\n- 18:00 🇪🇸🇦🇩...,"[LeonesDeMostoles, KingsLeagueJ5]",UltMostoles
1499,2023-02-05 04:02:22+00:00,#KingsLeagueJ5,[KingsLeagueJ5],MelvinYerena
1500,2023-02-05 02:45:53+00:00,🎪Vuelve la #KingsLeague🎪 \n\n➡️Veremos las pri...,"[KingsLeague, KingsLeagueJ5]",luigisanabria


In [ ]:
#Extraemos los hashtags para convertirlos en strings
def textl(text):
  leters= " ".join([str(_) for _ in text])
  return leters


In [ ]:
df['Token'] = df['Hashtag'].apply(textl)
df.head()

<ipython-input-17-0693a8dedba9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Token'] = df['Hashtag'].apply(textl)


,Date,Tweet,Hashtag,Username,Token
85,2023-02-05 23:51:27+00:00,XBUYER TEAM de XBUYER VS 1K FC de IKER CASILLA...,[KingsLeagueJ5],jorgedelcuri,KingsLeagueJ5
86,2023-02-05 23:50:07+00:00,"Que @IbaiLlanos , ni @auronplay , los directos...",[KingsLeagueJ5],dairony26,KingsLeagueJ5
87,2023-02-05 23:32:18+00:00,Aniquiladores está para campeón?? #KingsLeagueJ5,[KingsLeagueJ5],Cristian_OuO,KingsLeagueJ5
88,2023-02-05 23:25:37+00:00,"¡Fin del programa!\n\nUn placer, como siempre ...",[KingsLeagueJ5],KioscoKL,KingsLeagueJ5
89,2023-02-05 23:24:48+00:00,💣 @Spursito hace SCOUTING de la Queens League ...,"[KingsLeagueJ5, QueensLeague]",KioscoKL,KingsLeagueJ5 QueensLeague


In [ ]:
tokenHashtag = df['Token']
tokenHashtag

85                                         KingsLeagueJ5
86                                         KingsLeagueJ5
87                                         KingsLeagueJ5
88                                         KingsLeagueJ5
89                            KingsLeagueJ5 QueensLeague
                              ...                       
1497                                       KingsLeaguej5
1498                      LeonesDeMostoles KingsLeagueJ5
1499                                       KingsLeagueJ5
1500                           KingsLeague KingsLeagueJ5
1501    ChupChupKings5 KingsLeagueJ5 KingsLeagueInfoJobs
Name: Token, Length: 1417, dtype: object

In [ ]:
tokens = [token for line in tokenHashtag  for token in line.split()]

In [ ]:
#Creamos una funcion que cuente los datos ya toquenizados 
def get_tokens(docx,num=40):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
tokensTotal = get_tokens(tokens)

In [ ]:
#Insertamos los datos en un dataframe
data = pd.DataFrame(tokensTotal.items(),columns=['Hashtags','scores'])
data

,Hashtags,scores
0,KingsLeagueJ5,1082
1,KingsLeague,305
2,KingsLeaguej5,265
3,VisionNoticias,97
4,InfoJobs,92
5,PUROPINCHEPIO,52
6,AniquiladoresFC,43
7,kingsleaguej5,36
8,PIOvsRAYO,33
9,KINGSLEAGUEJ5,29


In [ ]:
#Ploteamos los datos
fig = px.bar(data, title="Hashtags mas usados de la Jornada 5 de la Kings League",x=data['Hashtags'], y=data['scores'])
fig.show()

#Conclusiones de los hashtags

Los hashtags mas utilizados son de la propia kingsLeague como pueden ser:


*   KingsLeagueJ5
*   KingsLeague
*   InfoJobs
*   KingsLeaguej5

Luego a nivel de losequipos que pertenecen a esta liga tenemos:


*   AniquiladoresFC
*   PioFC
*   LosTroncosFC
*   ElBarrioFc










# Analisis de emociones de los tweets

In [ ]:
#Removemos caracteres que puedan afectar la prediccion del modelo
df['Tweet'] = df['Tweet'].apply(nt.remove_hashtags)
df['Tweet'] = df['Tweet'].apply(nt.remove_emojis)
df['Tweet'] = df['Tweet'].apply(nt.remove_numbers)
df['Tweet'] = df['Tweet'].apply(nt.remove_special_characters)
df['Tweet'] = df['Tweet'].apply(nt.remove_urls)
df['Tweet'] = df['Tweet'].apply(nt.remove_currency_symbols)
df['Tweet'] = df['Tweet'].apply(nt.remove_userhandles)

In [ ]:
#Creamos una funcion para borrar caracteres especiales
def clean_text(text):
  text = re.sub(r'@[A-Za-z09]+', '', text)
  text = re.sub(r'#', '', text)
  text = re.sub(r'RT[\s]+', '', text)
  text = re.sub(r'https?:\/\/?', '', text)


  return text

df['Tweet'] = df['Tweet'].apply(clean_text)

In [ ]:
#Creamos dos variables que analicen los sentimientos y emociones de los tweets
analyzer = create_analyzer(task="sentiment", lang="es")
emotion_analyzer = create_analyzer(task="emotion", lang="es")

In [ ]:
#Creamos dos funciones para el analisis 
def get_sentiment(Text):
  return analyzer.predict(Text).output

def get_emotion(Text):
  return emotion_analyzer.predict(Text).output


In [ ]:
df['Sentiment'] = df['Tweet'].apply(get_sentiment)

<ipython-input-29-c1634b6b7ced>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df['Emotion'] = df['Tweet'].apply(get_emotion)

<ipython-input-30-e23d33ea5937>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Contamos las variables
count = df['Sentiment'].value_counts()
count

NEU    786
NEG    385
POS    246
Name: Sentiment, dtype: int64

In [ ]:
fig = px.bar(count, title="Sentimientos de los tweets de la kings league jornada 5", color=['red','blue','green'])
fig.show()

#Conclusiones de el analisis de emociones

Podemos concluir que hay mas comentarios negativos que positivos pero lo que mas se resalta es los comentarios neutros con una alta diferencia.



1.   Neutros: 786
2.   Negativos: 385
3.   Positivos: 246





In [ ]:
# Creamos tres sets de datos con los diferentes tipos de comentarios
positive_tweet = df[df['Sentiment'] == 'POS']
neutral_tweet = df[df['Sentiment'] == 'NEU']
negative_tweet = df[df['Sentiment'] == 'NEG']

In [ ]:
print(stopwords.words('spanish'))

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [ ]:
#Eliminamos las preposiciones, adverbios y adjetivos
def textdrop(text):

  sw = stopwords.words('spanish')

  text = [word.lower() for word in text.split() if word.lower() not in sw]

  return " ".join(text)

In [ ]:
positive_tweet['Tweet'] = positive_tweet['Tweet'].apply(textdrop)
neutral_tweet['Tweet'] = neutral_tweet['Tweet'].apply(textdrop)
negative_tweet['Tweet'] = negative_tweet['Tweet'].apply(textdrop)

In [ ]:
tokenPositive = positive_tweet['Tweet']
tokenNeutral = neutral_tweet['Tweet']
tokenNegative = negative_tweet['Tweet']

In [ ]:
#Tokenizamos los tweets
pos_tokens = [token for line in tokenPositive  for token in line.split()]
neg_tokens = [token for line in tokenNeutral  for token in line.split()]
neut_tokens = [token for line in tokenNegative  for token in line.split()]

In [ ]:
word_pos = get_tokens(pos_tokens)
word_neu = get_tokens(neut_tokens)
word_neg = get_tokens(neg_tokens)

In [ ]:
#Creamos 3 sets de datos con las palabras y el contador 
pos_df = pd.DataFrame(word_pos.items(),columns=['words','scores'])
neut_df = pd.DataFrame(word_neu.items(),columns=['words','scores'])
neg_df = pd.DataFrame(word_neg.items(),columns=['words','scores'])

In [ ]:
fig = px.bar(data, title="Palabras mas usadas en los tweets positivos de la Jornada 5 de la Kings League",x=pos_df['words'], y=pos_df['scores'])
fig.show()

In [ ]:
fig = px.bar(data, title="Palabras mas usadas en los tweets negativos de la Jornada 5 de la Kings League",x=neg_df['words'], y=neg_df['scores'])
fig.show()

#Conclusiones de la tokenzacion

Podemos observar que en los tweets positivos las palabras claves que se repiten comunmente es:

*   Kingsleague
*   TheGrefg presindentes saiyans
*   LosTroncosFc
*   Aniquiladores

En los tweets negativo los mas resaltantes son:

*   JijantesFc
*   LosTroncosFc
*   PioFc
*   1KFutbolClub











In [ ]:
fig = px.histogram(df, x=df["Date"], title="Tweets durante durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

#Tweets durante la Kings League

. Podemos ver que en la previa del partido hubo una subida de tweets a las 3:30 de la tarde hora españa.

. Podemos observar que hubo una subida de tweets luego de terminar el partido de XBuyer vs 1K.

. A las 5 empieza el SaiyansFc vs El Barrio se generan 57 tweets y cuando termina se concluye a la 5:30 con 86 tweets.

. El Kunisports vs Ultimate Mostoles empezo a las 6 con 105 tweets y al terminar se concluye a las 6:30 con 124 tweets.

. El JijantesFc vs PorcinosFc empezo a las 7 con 110 tweets y al terminar se concluye a las 7:30 con 141 tweets.

. El Rayo de Barcelona vs PioFc empezo a las 8 con 116 tweets y al terminar se concluye a las 8:30 con 167 tweets.


In [ ]:
fig = px.histogram(df, x=positive_tweet["Date"], title="Tweets positivos durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

#Pico de tweets positivos



*   El pico mas al es a las 9 con 50 tweets
*   El segundo pico mas alto es a las 6 con 44 tweets



In [ ]:
fig = px.histogram(df, x=negative_tweet["Date"], title="Tweets negativos durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

#Pico de tweets positivos



*   El pico mas al es a las 7:30 con 62 tweets
*   El segundo pico mas alto es a las 8:30 con 57 tweets